In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import pickle as pkl

pd.set_option('display.max_columns', 1000)

In [2]:
teams_df = pd.read_csv('wc_team_rosters.csv')
teams_df.head()

,Country,CountryID,group_id,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,player_11
0,Qatar,0,1,180548,268781,268782,268771,238878,239861,268778,215911,180561,268772,234051
1,Ecuador,1,1,253354,246055,252053,256197,237942,247181,246057,256079,241498,220295,240944
2,Senegal,2,1,234642,215699,201024,10530,225711,186801,198861,238227,193474,235353,246242
3,Netherlands,3,1,162328,233096,251805,203376,208920,190815,228702,200260,203483,225953,202556
4,England,4,2,204935,188377,203574,203263,229984,224081,234378,233064,237692,202126,202652


In [3]:
players_df = pd.read_csv('wc_player_stats.csv')
players_df.head()

,player_id,Overall,Potential,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Defensive Awareness,Standing Tackle,Sliding Tackle,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Shot Power,Jumping,Stamina,Strength,Long Shots,Acceleration,Sprint Speed,Agility,Reactions,Balance,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys
0,180548,68,68,71,65,68,66,65,13,13,12,38,12,11,51,10,15,51,60,37,50,9,50,44,40,66,30,17,13,9,15,20,11,12,14,16,11
1,268781,69,69,11,10,15,7,9,64,65,69,71,66,58,59,33,62,45,65,83,77,34,72,75,61,59,57,60,49,35,64,64,65,35,58,61,29
2,268782,68,75,8,8,9,14,9,65,71,68,75,62,42,55,45,65,63,88,68,74,57,73,71,74,58,75,61,72,73,62,65,49,38,63,65,40
3,268771,71,71,9,14,6,14,7,72,70,67,68,68,46,50,71,68,75,85,77,79,72,58,65,47,67,57,48,64,63,64,61,39,49,74,66,46
4,238878,66,74,7,14,12,9,15,65,67,64,65,62,45,60,35,67,30,77,75,71,39,73,75,60,68,67,53,61,40,66,63,39,31,62,67,42


In [4]:
# rename columns to make compatible with namings for European Soccer Dataset
players_df = players_df.rename({'Overall': 'overall_rating'},axis=1)
players_df.columns = ['_'.join(col.lower().split(' ')) for col in players_df.columns]
players_df.head()

,player_id,overall_rating,potential,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,defensive_awareness,standing_tackle,sliding_tackle,aggression,interceptions,positioning,vision,penalties,composure,shot_power,jumping,stamina,strength,long_shots,acceleration,sprint_speed,agility,reactions,balance,dribbling,curve,fk_accuracy,long_passing,ball_control,crossing,finishing,heading_accuracy,short_passing,volleys
0,180548,68,68,71,65,68,66,65,13,13,12,38,12,11,51,10,15,51,60,37,50,9,50,44,40,66,30,17,13,9,15,20,11,12,14,16,11
1,268781,69,69,11,10,15,7,9,64,65,69,71,66,58,59,33,62,45,65,83,77,34,72,75,61,59,57,60,49,35,64,64,65,35,58,61,29
2,268782,68,75,8,8,9,14,9,65,71,68,75,62,42,55,45,65,63,88,68,74,57,73,71,74,58,75,61,72,73,62,65,49,38,63,65,40
3,268771,71,71,9,14,6,14,7,72,70,67,68,68,46,50,71,68,75,85,77,79,72,58,65,47,67,57,48,64,63,64,61,39,49,74,66,46
4,238878,66,74,7,14,12,9,15,65,67,64,65,62,45,60,35,67,30,77,75,71,39,73,75,60,68,67,53,61,40,66,63,39,31,62,67,42


In [5]:
groups = teams_df.groupby('group_id')
matches = []
for group_idx, group_df in groups:
    # create group matchups
    for i in range(len(group_df)):
        for j in range(i + 1, len(group_df)):
            match = list(group_df.iloc[i]) + list(group_df.iloc[j])
            matches.append(match)
columns = ['Country_1', 'CountryID_1', 'GroupID_1'] + [f'player_{i}_1' for i in range(1, 12)]
columns += ['Country_2', 'CountryID_2', 'GroupID_2'] + [f'player_{i}_2' for i in range(1, 12)]
group_matches_df = pd.DataFrame(matches, columns=columns)
group_matches_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2
0,Qatar,0,1,180548,268781,268782,268771,238878,239861,268778,215911,180561,268772,234051,Ecuador,1,1,253354,246055,252053,256197,237942,247181,246057,256079,241498,220295,240944
1,Qatar,0,1,180548,268781,268782,268771,238878,239861,268778,215911,180561,268772,234051,Senegal,2,1,234642,215699,201024,10530,225711,186801,198861,238227,193474,235353,246242
2,Qatar,0,1,180548,268781,268782,268771,238878,239861,268778,215911,180561,268772,234051,Netherlands,3,1,162328,233096,251805,203376,208920,190815,228702,200260,203483,225953,202556
3,Ecuador,1,1,253354,246055,252053,256197,237942,247181,246057,256079,241498,220295,240944,Senegal,2,1,234642,215699,201024,10530,225711,186801,198861,238227,193474,235353,246242
4,Ecuador,1,1,253354,246055,252053,256197,237942,247181,246057,256079,241498,220295,240944,Netherlands,3,1,162328,233096,251805,203376,208920,190815,228702,200260,203483,225953,202556


In [6]:
group_matches_df.shape

(48, 28)

In [7]:
def merge_team_stats(matches_df, team_idx, suffix):
    for i in range(1, 12):
        matches_df = matches_df.merge(players_df, left_on=[f'player_{i}_{team_idx}'], right_on=['player_id'], how='left', suffixes=[None, f'_{suffix}_{i}'])
    return matches_df

In [9]:
def rename_columns(matches_df, suffix):
    for col in matches_df.columns:
        if col in players_df.columns:
    #         print(col)
            matches_df.rename(columns = {col:f'{col}_{suffix}_1'},inplace=True)
    return matches_df

# group_matches_df = group_matches_df.rename(rename_dict, axis=1)

In [10]:
group_matches_df = merge_team_stats(group_matches_df, 1, 'home')
group_matches_df = rename_columns(group_matches_df, 'home')
group_matches_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [11]:
group_matches_df = merge_team_stats(group_matches_df, 2, 'away')
group_matches_df = rename_columns(group_matches_df, 'away')
group_matches_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [12]:
selected_cols = ['potential_home_1',
 'potential_home_2',
 'overall_rating_home_3',
 'overall_rating_home_4',
 'overall_rating_home_5',
 'ball_control_home_5',
 'overall_rating_home_6',
 'potential_home_6',
 'overall_rating_home_7',
 'overall_rating_home_8',
 'overall_rating_home_9',
 'positioning_home_10',
 'overall_rating_home_11',
 'overall_rating_away_1',
 'overall_rating_away_2',
 'overall_rating_away_3',
 'potential_away_3',
 'overall_rating_away_4',
 'standing_tackle_away_4',
 'overall_rating_away_5',
 'overall_rating_away_6',
 'overall_rating_away_7',
 'overall_rating_away_8',
 'overall_rating_away_9',
 'positioning_away_10',
 'overall_rating_away_11',
 'potential_away_11'
]

In [13]:
input_df = group_matches_df[selected_cols]

In [14]:
model_xgb = pkl.load(open('model_xgb.pkl', 'rb'))

In [15]:
preds = model_xgb.predict(input_df.to_numpy())

In [16]:
group_matches_df['predictions'] = preds
print(group_matches_df[['Country_1', 'Country_2', 'predictions']])

       Country_1     Country_2  predictions
0          Qatar       Ecuador            2
1          Qatar       Senegal            2
2          Qatar   Netherlands            2
3        Ecuador       Senegal            2
4        Ecuador   Netherlands            2
5        Senegal   Netherlands            2
6        England          Iran            1
7        England           USA            1
8        England         Wales            1
9           Iran           USA            2
10          Iran         Wales            2
11           USA         Wales            1
12     Argentina  Saudi Arabia            1
13     Argentina        Mexico            1
14     Argentina        Poland            1
15  Saudi Arabia        Mexico            2
16  Saudi Arabia        Poland            2
17        Mexico        Poland            1
18        France     Australia            1
19        France       Denmark            1
20        France       Tunisia            1
21     Australia       Denmark  

In [17]:
group_stage_winners = [['Netherlands', 'Senegal'], ['England', 'USA'], ['Argentina', 'Mexico'], 
                       ['France', 'Denmark'], ['Germany', 'Spain'], ['Belgium', 'Croatia'], 
                       ['Brazil', 'Serbia'], ['Portugal', 'Uruguay']]
round_16_matches = []
for i in range(0, len(group_stage_winners), 2):
    team_1 = teams_df.loc[teams_df['Country'] == group_stage_winners[i][0]].iloc[0]
    team_2 = teams_df.loc[teams_df['Country'] == group_stage_winners[i+1][1]].iloc[0]
    row = list(team_1) + list(team_2)
    round_16_matches.append(row)
    # repeat for second of first group and first of second group
    team_1 = teams_df.loc[teams_df['Country'] == group_stage_winners[i][1]].iloc[0]
    team_2 = teams_df.loc[teams_df['Country'] == group_stage_winners[i+1][0]].iloc[0]
    row = list(team_1) + list(team_2)
    round_16_matches.append(row)

columns = ['Country_1', 'CountryID_1', 'GroupID_1'] + [f'player_{i}_1' for i in range(1, 12)]
columns += ['Country_2', 'CountryID_2', 'GroupID_2'] + [f'player_{i}_2' for i in range(1, 12)]
round_16_matches_df = pd.DataFrame(round_16_matches, columns=columns)

In [18]:
round_16_matches_df.shape

(8, 28)

In [19]:
round_16_matches_df

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2
0,Netherlands,3,1,162328,233096,251805,203376,208920,190815,228702,200260,203483,225953,202556,USA,6,2,233267,251804,212591,222123,229348,232999,238744,247361,241496,241487,227796
1,Senegal,2,1,234642,215699,201024,10530,225711,186801,198861,238227,193474,235353,246242,England,4,2,204935,188377,203574,203263,229984,224081,234378,233064,237692,202126,202652
2,Argentina,8,3,202811,233084,232488,192366,224334,207439,212616,226226,158023,231478,183898,Denmark,14,4,163587,172522,213661,179645,234678,213648,193283,190460,240017,228687,241508
3,Mexico,10,3,140233,233493,229980,173432,226045,235844,235389,171897,233260,204838,221992,France,12,4,167948,241486,201535,225850,241637,195864,213345,232656,194765,165153,231747
4,Germany,16,5,167495,226229,212190,205452,236703,186942,212622,210324,222492,189596,212188,Croatia,23,6,241671,257771,195086,251517,243388,216352,223273,177003,207410,181458,216354
5,Spain,19,5,230869,204963,245037,212218,189332,189511,193747,251854,198950,201153,241461,Belgium,20,6,192119,184087,220182,172871,202371,216393,192985,208418,200155,183277,192505
6,Brazil,24,7,230182,230201,230375,230225,230256,230258,230347,230271,230481,230430,230294,Uruguay,31,8,223690,219914,182493,216460,240716,227535,239053,219985,253072,253283,176580
7,Serbia,26,7,241727,238095,209547,254840,221354,208574,193198,239433,193881,215716,199434,Portugal,30,8,178005,210514,120533,239818,209889,224293,207566,212198,218667,20801,224458


In [20]:
round_16_matches_df = merge_team_stats(round_16_matches_df, 1, 'home')
round_16_matches_df = rename_columns(round_16_matches_df, 'home')
round_16_matches_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [21]:
round_16_matches_df = merge_team_stats(round_16_matches_df, 2, 'away')
round_16_matches_df = rename_columns(round_16_matches_df, 'away')
round_16_matches_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [23]:
input_df = round_16_matches_df[selected_cols]
preds = model_xgb.predict(input_df.to_numpy())
round_16_matches_df['predictions'] = preds
print(round_16_matches_df[['Country_1', 'Country_2', 'predictions']])

     Country_1 Country_2  predictions
0  Netherlands       USA            1
1      Senegal   England            2
2    Argentina   Denmark            1
3       Mexico    France            2
4      Germany   Croatia            1
5        Spain   Belgium            1
6       Brazil   Uruguay            1
7       Serbia  Portugal            2


In [49]:
quarter_final_matches = [['Argentina', 'Netherlands'], ['England', 'France'], ['Brazil', 'Germany'], ['Spain', 'Portugal']]
quarter_finals = []
for i in range(0, len(quarter_final_matches)):
    team_1 = teams_df.loc[teams_df['Country'] == quarter_final_matches[i][0]].iloc[0]
    team_2 = teams_df.loc[teams_df['Country'] == quarter_final_matches[i][1]].iloc[0]
    row = list(team_1) + list(team_2)
    quarter_finals.append(row)

columns = ['Country_1', 'CountryID_1', 'GroupID_1'] + [f'player_{i}_1' for i in range(1, 12)]
columns += ['Country_2', 'CountryID_2', 'GroupID_2'] + [f'player_{i}_2' for i in range(1, 12)]
quarter_finals_df = pd.DataFrame(quarter_finals, columns=columns)

In [50]:
quarter_finals_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2
0,Argentina,8,3,202811,233084,232488,192366,224334,207439,212616,226226,158023,231478,183898,Netherlands,3,1,162328,233096,251805,203376,208920,190815,228702,200260,203483,225953,202556
1,England,4,2,204935,188377,203574,203263,229984,224081,234378,233064,237692,202126,202652,France,12,4,167948,241486,201535,225850,241637,195864,213345,232656,194765,165153,231747
2,Brazil,24,7,230182,230201,230375,230225,230256,230258,230347,230271,230481,230430,230294,Germany,16,5,167495,226229,212190,205452,236703,186942,212622,210324,222492,189596,212188
3,Spain,19,5,230869,204963,245037,212218,189332,189511,193747,251854,198950,201153,241461,Portugal,30,8,178005,210514,120533,239818,209889,224293,207566,212198,218667,20801,224458


In [51]:
quarter_finals_df = merge_team_stats(quarter_finals_df, 1, 'home')
quarter_finals_df = rename_columns(quarter_finals_df, 'home')
quarter_finals_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [52]:
quarter_finals_df = merge_team_stats(quarter_finals_df, 2, 'away')
quarter_finals_df = rename_columns(quarter_finals_df, 'away')
quarter_finals_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [53]:
input_df = quarter_finals_df[selected_cols]
preds = model_xgb.predict(input_df.to_numpy())
quarter_finals_df['predictions'] = preds
print(quarter_finals_df[['Country_1', 'Country_2', 'predictions']])

   Country_1    Country_2  predictions
0  Argentina  Netherlands            1
1    England       France            1
2     Brazil      Germany            2
3      Spain     Portugal            2


In [39]:
semi_final_matches = [['Argentina', 'Germany'], ['England', 'Portugal']]
semi_finals = []
for i in range(0, len(semi_final_matches)):
    team_1 = teams_df.loc[teams_df['Country'] == semi_final_matches[i][0]].iloc[0]
    team_2 = teams_df.loc[teams_df['Country'] == semi_final_matches[i][1]].iloc[0]
    row = list(team_1) + list(team_2)
    semi_finals.append(row)

columns = ['Country_1', 'CountryID_1', 'GroupID_1'] + [f'player_{i}_1' for i in range(1, 12)]
columns += ['Country_2', 'CountryID_2', 'GroupID_2'] + [f'player_{i}_2' for i in range(1, 12)]
semi_finals_df = pd.DataFrame(semi_finals, columns=columns)

In [40]:
semi_finals_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2
0,Argentina,8,3,202811,233084,232488,192366,224334,207439,212616,226226,158023,231478,183898,Germany,16,5,167495,226229,212190,205452,236703,186942,212622,210324,222492,189596,212188
1,England,4,2,204935,188377,203574,203263,229984,224081,234378,233064,237692,202126,202652,Portugal,30,8,178005,210514,120533,239818,209889,224293,207566,212198,218667,20801,224458


In [41]:
semi_finals_df = merge_team_stats(semi_finals_df, 1, 'home')
semi_finals_df = rename_columns(semi_finals_df, 'home')
semi_finals_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [42]:
semi_finals_df = merge_team_stats(semi_finals_df, 2, 'away')
semi_finals_df = rename_columns(semi_finals_df, 'away')
semi_finals_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [43]:
input_df = semi_finals_df[selected_cols]
preds = model_xgb.predict(input_df.to_numpy())
semi_finals_df['predictions'] = preds
print(semi_finals_df[['Country_1', 'Country_2', 'predictions']])

   Country_1 Country_2  predictions
0  Argentina   Germany            1
1    England  Portugal            2


In [44]:
final = ['Argentina', 'Portugal']
team_1 = teams_df.loc[teams_df['Country'] == final[0]].iloc[0]
team_2 = teams_df.loc[teams_df['Country'] == final[1]].iloc[0]
row = list(team_1) + list(team_2)
columns = ['Country_1', 'CountryID_1', 'GroupID_1'] + [f'player_{i}_1' for i in range(1, 12)]
columns += ['Country_2', 'CountryID_2', 'GroupID_2'] + [f'player_{i}_2' for i in range(1, 12)]
final_df = pd.DataFrame([row], columns=columns)

In [45]:
final_df

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2
0,Argentina,8,3,202811,233084,232488,192366,224334,207439,212616,226226,158023,231478,183898,Portugal,30,8,178005,210514,120533,239818,209889,224293,207566,212198,218667,20801,224458


In [46]:
final_df = merge_team_stats(final_df, 1, 'home')
final_df = rename_columns(final_df, 'home')
final_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [47]:
final_df = merge_team_stats(final_df, 2, 'away')
final_df = rename_columns(final_df, 'away')
final_df.head()

,Country_1,CountryID_1,GroupID_1,player_1_1,player_2_1,player_3_1,player_4_1,player_5_1,player_6_1,player_7_1,player_8_1,player_9_1,player_10_1,player_11_1,Country_2,CountryID_2,GroupID_2,player_1_2,player_2_2,player_3_2,player_4_2,player_5_2,player_6_2,player_7_2,player_8_2,player_9_2,player_10_2,player_11_2,player_id_home_1,overall_rating_home_1,potential_home_1,gk_diving_home_1,gk_handling_home_1,gk_kicking_home_1,gk_positioning_home_1,gk_reflexes_home_1,defensive_awareness_home_1,standing_tackle_home_1,sliding_tackle_home_1,aggression_home_1,interceptions_home_1,positioning_home_1,vision_home_1,penalties_home_1,composure_home_1,shot_power_home_1,jumping_home_1,stamina_home_1,strength_home_1,long_shots_home_1,acceleration_home_1,sprint_speed_home_1,agility_home_1,reactions_home_1,balance_home_1,dribbling_home_1,curve_home_1,fk_accuracy_home_1,long_passing_home_1,ball_control_home_1,crossing_home_1,finishing_home_1,heading_accuracy_home_1,short_passing_home_1,volleys_home_1,player_id_home_2,overall_rating_home_2,potential_home_2,gk_diving_home_2,gk_handling_home_2,gk_kicking_home_2,gk_positioning_home_2,gk_reflexes_home_2,defensive_awareness_home_2,standing_tackle_home_2,sliding_tackle_home_2,aggression_home_2,interceptions_home_2,positioning_home_2,vision_home_2,penalties_home_2,composure_home_2,shot_power_home_2,jumping_home_2,stamina_home_2,strength_home_2,long_shots_home_2,acceleration_home_2,sprint_speed_home_2,agility_home_2,reactions_home_2,balance_home_2,dribbling_home_2,curve_home_2,fk_accuracy_home_2,long_passing_home_2,ball_control_home_2,crossing_home_2,finishing_home_2,heading_accuracy_home_2,short_passing_home_2,volleys_home_2,player_id_home_3,overall_rating_home_3,potential_home_3,gk_diving_home_3,gk_handling_home_3,gk_kicking_home_3,gk_positioning_home_3,gk_reflexes_home_3,defensive_awareness_home_3,standing_tackle_home_3,sliding_tackle_home_3,aggression_home_3,interceptions_home_3,positioning_home_3,vision_home_3,penalties_home_3,composure_home_3,shot_power_home_3,jumping_home_3,stamina_home_3,strength_home_3,long_shots_home_3,acceleration_home_3,sprint_speed_home_3,agility_home_3,reactions_home_3,balance_home_3,dribbling_home_3,curve_home_3,fk_accuracy_home_3,long_passing_home_3,ball_control_home_3,crossing_home_3,finishing_home_3,heading_accuracy_home_3,short_passing_home_3,volleys_home_3,player_id_home_4,overall_rating_home_4,potential_home_4,gk_diving_home_4,gk_handling_home_4,gk_kicking_home_4,gk_positioning_home_4,gk_reflexes_home_4,defensive_awareness_home_4,standing_tackle_home_4,sliding_tackle_home_4,aggression_home_4,interceptions_home_4,positioning_home_4,vision_home_4,penalties_home_4,composure_home_4,shot_power_home_4,jumping_home_4,stamina_home_4,strength_home_4,long_shots_home_4,acceleration_home_4,sprint_speed_home_4,agility_home_4,reactions_home_4,balance_home_4,dribbling_home_4,curve_home_4,fk_accuracy_home_4,long_passing_home_4,ball_control_home_4,crossing_home_4,finishing_home_4,heading_accuracy_home_4,short_passing_home_4,volleys_home_4,player_id_home_5,overall_rating_home_5,potential_home_5,gk_diving_home_5,gk_handling_home_5,gk_kicking_home_5,gk_positioning_home_5,gk_reflexes_home_5,defensive_awareness_home_5,standing_tackle_home_5,sliding_tackle_home_5,aggression_home_5,interceptions_home_5,positioning_home_5,vision_home_5,penalties_home_5,composure_home_5,shot_power_home_5,jumping_home_5,stamina_home_5,strength_home_5,long_shots_home_5,acceleration_home_5,sprint_speed_home_5,agility_home_5,reactions_home_5,balance_home_5,dribbling_home_5,curve_home_5,fk_accuracy_home_5,long_passing_home_5,ball_control_home_5,crossing_home_5,finishing_home_5,heading_accuracy_home_5,short_passing_home_5,volleys_home_5,player_id_home_6,overall_rating_home_6,potential_home_6,gk_diving_home_6,gk_handling_home_6,gk_kicking_home_6,gk_positioning_home_6,gk_reflexes_home_6,defensive_awareness_home_6,standing_tackle_home_6,sliding_tackle_home_6,aggression_home_6,interceptions_home_6,positioning_home_6,vision_h

In [48]:
input_df = final_df[selected_cols]
preds = model_xgb.predict(input_df.to_numpy())
final_df['predictions'] = preds
print(final_df[['Country_1', 'Country_2', 'predictions']])

   Country_1 Country_2  predictions
0  Argentina  Portugal            2
